In [1]:
# GO/KEGG Enrichment Analysis for Alternaria alternata (Non_model_org)

In [2]:
"""
Non model organism: Alternaria alternata
Model organism: Aspergillus nidulans

It is assume that you've obtained your blast results. (A.alternata against A.nidulans)

Step 1: Obtain A.alternata annotation file and map the transcript ids to Gene ids in the DEGs
Step 2: Obtain A.nidulans annotation file and map the Gene ids to the protein ids in the blast output
Step 3: Map transcript ids in DEGs to transcript ids in blast resultsA.nidulans gene ids
Step 4: Filter using the transcript ids for your gene of interest

"""


"\nNon model organism: Alternaria alternata\nModel organism: Aspergillus nidulans\n\nIt is assume that you've obtained your blast results. (A.alternata against A.nidulans)\n\nStep 1: Obtain A.alternata annotation file and map the transcript ids to Gene ids in the DEGs\nStep 2: Obtain A.nidulans annotation file and map the Gene ids to the protein ids in the blast output\nStep 3: Map transcript ids in DEGs to transcript ids in blast resultsA.nidulans gene ids\nStep 4: Filter using the transcript ids for your gene of interest\n\n"

In [3]:
import pandas as pd

In [4]:
# Extract transcript ids and gene ids from the annotation file
# Source: Alternaria gtf file from Ensembl

def get_ens_dict(file_path):
    with open(file_path) as f:
        gtf = list(f)

    gtf = [x for x in gtf if not x.startswith('#')]
    gtf = [x for x in gtf if 'gene_id "' in x and 'gene_name "' in x]
    if len(gtf) == 0:
        print('change gene_id " and gene_name " formats if necessary')
    
    gtf = list(map(lambda x: (x.split('gene_id "')[1].split('"')[0], x.split('gene_name "')[1].split('"')[0]), gtf))
    gtf = dict(set(gtf))
    return gtf

gtf_dict = get_ens_dict('Alternaria_alternata_gca_004154755.ASM415475v1.60.gtf')

change gene_id " and gene_name " formats if necessary


In [5]:
# Open annotation file and extract both the transcript ids and gene ids

with open('Alternaria_alternata_gca_004154755.ASM415475v1.60.gtf') as f:
    gtf = list(f)

In [6]:
gtf = [x for x in gtf if not x.startswith('#')]
gtf = [x for x in gtf if 'gene_id "' in x and 'transcript_id "' in x]

In [7]:
gtf[0]

'contig_1\tena\ttranscript\t1506662\t1507296\t.\t+\t.\tgene_id "AA0117_g585"; transcript_id "RYN84190"; gene_source "ena"; gene_biotype "protein_coding"; transcript_source "ena"; transcript_biotype "protein_coding"; tag "Ensembl_canonical";\n'

In [8]:
gtf = list(map(lambda x: (x.split('gene_id "')[1].split('"')[0], x.split('transcript_id "')[1].split('"')[0]), gtf))
gtf = list(set(gtf))

In [9]:
len(gtf)

13640

In [10]:
gtf[:5]

[('AA0117_g4461', 'RYN78396'),
 ('AA0117_g10291', 'RYN70738'),
 ('AA0117_g11775', 'RYN66750'),
 ('AA0117_g13378', 'RYN52004'),
 ('AA0117_g12513', 'RYN64386')]

In [11]:
gtf = list(set(gtf))

In [12]:
len(gtf)

13640

In [13]:
# Load differential expression results (DEGs)
alternaria_degs = pd.read_csv("sigs_expreesed_genes_alternaria_14dpi.csv")
alternaria_degs.head()

,Geneid,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,AA0117_g1326,17.557861,7.364455,1.897538,3.881057,0.000104,0.000886
1,AA0117_g510,10.128432,6.570855,2.045535,3.212292,0.001317,0.005108
2,AA0117_g1759,10.614220,6.638520,1.918973,3.459412,0.000541,0.002890
3,AA0117_g954,33.284664,8.287302,1.806274,4.588065,0.000004,0.000107
4,AA0117_g1218,11.968501,6.811507,2.080628,3.273775,0.001061,0.004476


In [14]:
gtf = dict(gtf)

In [16]:
alternaria_degs['TranscriptID'] = alternaria_degs['Geneid'].map(gtf)

In [17]:
alternaria_degs

,Geneid,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,TranscriptID
0,AA0117_g1326,17.557861,7.364455,1.897538,3.881057,0.000104,0.000886,RYN84069
1,AA0117_g510,10.128432,6.570855,2.045535,3.212292,0.001317,0.005108,RYN83116
2,AA0117_g1759,10.614220,6.638520,1.918973,3.459412,0.000541,0.002890,RYN83339
3,AA0117_g954,33.284664,8.287302,1.806274,4.588065,0.000004,0.000107,RYN84121
4,AA0117_g1218,11.968501,6.811507,2.080628,3.273775,0.001061,0.004476,RYN84463
...,...,...,...,...,...,...,...,...
1150,AA0117_g12602,7.595864,6.155378,2.068754,2.975403,0.002926,0.008543,RYN63955
1151,AA0117_g12613,19.017888,7.479745,2.189190,3.416672,0.000634,0.003130,RYN63948
1152,AA0117_g12741,9.590966,6.491989,2.077977,3.124187,0.001783,0.006156,RYN63501
1153,AA0117_g12754,14.139050,7.051906,1.954599,3.607854,0.000309,0.001936,RYN63461


In [18]:
# List of DEGs
# Define up and down-regulated genes
def get_regulated_genes(de_results, log2fc_threshold=1.5):
    up_genes = de_results[de_results['log2FoldChange'] > log2fc_threshold]['TranscriptID'].tolist()
    down_genes = de_results[de_results['log2FoldChange'] < -log2fc_threshold]['TranscriptID'].tolist()
    return up_genes, down_genes

In [19]:
up_regulated_genes, down_regulated_genes = get_regulated_genes(alternaria_degs)

In [20]:
print(f"Number of up regulated genes: {len(up_regulated_genes)}")

Number of up regulated genes: 1155


In [21]:
up_regulated_genes[:5]

['RYN84069', 'RYN83116', 'RYN83339', 'RYN84121', 'RYN84463']

In [22]:
print(f"Number of down regulated genes: {len(down_regulated_genes)}")

Number of down regulated genes: 0


In [23]:
# Load the annotation file (GFF3) from A.nidulans
gff_file = "aspergillus_nidulans.gff3"
gff_data = pd.read_csv(gff_file, sep="\t", comment="#", header=None)

In [24]:
# Extract protein-to-gene mapping
protein_to_gene = {}
for _, row in gff_data.iterrows():
    if row[2] == "mRNA":  # Look for mRNA entries
        attributes = row[8].split(";")
        protein_id = None
        gene_id = None
        for attr in attributes:
            if "ID=" in attr:
                protein_id = attr.split("=")[1]
            if "Parent=" in attr:
                gene_id = attr.split("=")[1]
        if protein_id and gene_id:
            protein_to_gene[protein_id] = gene_id

In [25]:
# Dataframe
protein_to_gene = pd.DataFrame(protein_to_gene.items(), columns=["ProteinID", "GeneID"])

In [26]:
# Clean up the protein_to_gene mapping
protein_to_gene["ProteinID"] = protein_to_gene["ProteinID"].str.replace("transcript:", "", regex=False)
protein_to_gene["GeneID"] = protein_to_gene["GeneID"].str.replace("gene:", "", regex=False)

In [27]:
protein_to_gene.head()

,ProteinID,GeneID
0,CBF69357,ANIA_10814
1,CBF69359,ANIA_10821
2,CBF69360,ANIA_06485
3,CBF69362,ANIA_06484
4,CBF69364,ANIA_10813


In [28]:
# Save the cleaned mapping
protein_to_gene.to_csv("protein_to_gene_mapping.csv", index=False)

In [29]:
# Load BLAST results
blast_results = pd.read_csv("blast_results.tsv", sep="\t", header=None)

In [30]:
blast_results.columns = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", 
    "qstart", "qend", "sstart", "send", "evalue", "bitscore"]

In [31]:
blast_results.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,RYN84190,CBF69991,27.957,186,77,5,38,166,39,224,2.600000e-14,66.6
1,RYN84069,CBF70428,84.286,140,19,3,1,140,1,137,8.450000e-82,236.0
2,RYN84069,CBF82379,26.866,67,49,0,81,147,765,831,5.200000e+00,25.8
3,RYN84069,CBF75082,23.585,106,71,2,17,122,138,233,6.400000e+00,25.8
4,RYN84069,CBF78087,35.294,34,20,1,53,84,446,479,9.700000e+00,25.0


In [32]:
# Merge BLAST results with the cleaned mapping
blast_results = pd.merge(blast_results, protein_to_gene, left_on="sseqid", right_on="ProteinID", how="left")

In [34]:
# Drop rows where no gene ID is found
blast_results = blast_results.dropna(subset=["GeneID"])

In [35]:
blast_results.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,ProteinID,GeneID
0,RYN84190,CBF69991,27.957,186,77,5,38,166,39,224,2.600000e-14,66.6,CBF69991,ANIA_06196
1,RYN84069,CBF70428,84.286,140,19,3,1,140,1,137,8.450000e-82,236.0,CBF70428,ANIA_05979
2,RYN84069,CBF82379,26.866,67,49,0,81,147,765,831,5.200000e+00,25.8,CBF82379,ANIA_09178
3,RYN84069,CBF75082,23.585,106,71,2,17,122,138,233,6.400000e+00,25.8,CBF75082,ANIA_03924
4,RYN84069,CBF78087,35.294,34,20,1,53,84,446,479,9.700000e+00,25.0,CBF78087,ANIA_08757


In [36]:
# Filter for DEGs in the BLAST results
blast_DEGs = blast_results[blast_results["qseqid"].isin(up_regulated_genes)]

In [37]:
blast_DEGs

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,ProteinID,GeneID
1,RYN84069,CBF70428,84.286,140,19,3,1,140,1,137,8.450000e-82,236.0,CBF70428,ANIA_05979
2,RYN84069,CBF82379,26.866,67,49,0,81,147,765,831,5.200000e+00,25.8,CBF82379,ANIA_09178
3,RYN84069,CBF75082,23.585,106,71,2,17,122,138,233,6.400000e+00,25.8,CBF75082,ANIA_03924
4,RYN84069,CBF78087,35.294,34,20,1,53,84,446,479,9.700000e+00,25.0,CBF78087,ANIA_08757
331,RYN83116,CBF87046,38.356,73,44,1,7,79,10,81,1.470000e-09,48.5,CBF87046,ANIA_02535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338063,RYN77530,CBF70234,57.143,301,87,4,1,298,1,262,5.920000e-114,328.0,CBF70234,ANIA_06077
338064,RYN77530,CBF77420,20.690,116,77,3,125,226,379,493,3.900000e-01,30.8,CBF77420,ANIA_04481
338065,RYN77530,CBF82133,25.444,169,103,7,69,225,13,170,5.300000e-01,30.4,CBF82133,ANIA_05305
338066,RYN77530,CBF84970,30.000,60,40,2,126,183,11,70,1.500000e+00,29.3,CBF84970,ANIA_01489


In [38]:
# Sort the DataFrame to prioritize top hits

In [39]:
# Convert evalue to numeric for sorting (scientific notation handling)
blast_DEGs['evalue'] = pd.to_numeric(blast_DEGs['evalue'], errors='coerce')

C:\Users\ukwun\AppData\Local\Temp\ipykernel_5936\3295821520.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blast_DEGs['evalue'] = pd.to_numeric(blast_DEGs['evalue'], errors='coerce')


In [40]:
df_sorted = blast_DEGs.sort_values(
    by=['qseqid', 'evalue', 'bitscore', 'pident'],
    ascending=[True, True, False, False]
)

In [41]:
# Drop duplicates to keep only the top hit per qseqid
top_hits_DEGs = df_sorted.drop_duplicates(subset='qseqid', keep='first')

In [42]:
top_hits_DEGs.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,ProteinID,GeneID
60806,RYN61308,CBF84128,75.000,428,90,6,1,427,1,412,0.000000e+00,584.0,CBF84128,ANIA_02731
329496,RYN63461,CBF73801,29.888,358,220,11,113,450,13,359,2.860000e-37,140.0,CBF73801,ANIA_08058
329347,RYN63501,CBF69923,40.153,391,193,7,164,515,1,389,4.460000e-90,280.0,CBF69923,ANIA_06220
320498,RYN63948,CBF71666,51.707,410,168,10,1,404,1,386,1.540000e-128,374.0,CBF71666,ANIA_06888
320270,RYN63955,CBF79317,58.104,1055,403,20,76,1111,91,1125,0.000000e+00,1135.0,CBF79317,ANIA_06986


In [43]:
len(top_hits_DEGs)

1155

In [44]:
# Save the top hits
top_hits_DEGs.to_csv('top_blast_hits_DEGs.csv', index=False)